In [1]:
#將特徵重要性較低的一半特徵刪除後，再做生存率預估，正確率是否有變化?
library(magrittr)
library(caret)
library(cattonum)
train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\titanic_train.csv')
str(train)
train=train[,-c(1,4,9)]
## Age NA --> mean 
age.na.loc=which(is.na(train$Age)==T)
train$Age[age.na.loc]=mean(train$Age,na.rm = T)

##extract categorical variable
col.type=rep(0,9)
for (i in 1:9) {
  col.type[i]=class(train[,i])
}
col.factor=which(col.type=='factor')

##only keep categorical column
train.cate=train[,c(col.factor,1)]
train.num=train[,-col.factor]

## covert NA into a level
for (j in 1:3) {
  train.cate[,j] %<>% addNA(.)
  
}

## fill somethong in empty cell
for (d in 1:3) {
  levels(train.cate[,d])[levels(train.cate[,d])=='']='empty'
}

## mean coding
mean.dat=catto_mean(train.cate,c('Sex','Cabin','Embarked'),response=Survived)

train.new=cbind(mean.dat[,-4],train.num[,-1],Survived=mean.dat[,4])


# 原始特徵 + 隨機森林
train_control <- trainControl(method="cv", number=5)
fit_rf = train(as.factor(Survived) ~ ., data=train.new, method='rf', trControl=train_control) # caret 自動處理文字型資料
impor.fea=varImp(fit_rf)

# 高重要性特徵(選一半) + 隨機森林
train.new.import=train.new[,c(1,2,5,8,9)]
fit_rf.important = train(as.factor(Survived) ~ ., data=train.new.import, method='rf', trControl=train_control) 

result=data.frame(type=c('rf','rf+f.select'),
                  acc=c(fit_rf$results[1,2],fit_rf.important$results[1,2]),
                  kappa=c(fit_rf$results[1,3],fit_rf.important$results[1,3]))
result




#將特徵重要性最高的兩個特徵做特徵組合，是否能再進一步提升預測力?

# sex + cabin
mean.dat=catto_mean(train.cate,c('Sex','Cabin'),response=Survived)
sex.cabin.mean=apply(mean.dat[,1:2],1,mean)

# 加入新重要性特徵 + 隨機森林
train.new.new=cbind(train.new,sex.cabin.mean)
fit_rf.new = train(as.factor(Survived) ~ ., data=train.new.new, method='rf', trControl=train_control) 

result.fin=data.frame(type=c('rf','rf+f.select','rf+new.f'),
                  acc=c(fit_rf$results[1,2],fit_rf.important$results[1,2],fit_rf.new$results[1,2]),
                  kappa=c(fit_rf$results[1,3],fit_rf.important$results[1,3],fit_rf.new$results[1,3]))
result.fin

Loading required package: lattice
Loading required package: ggplot2


ERROR: Error in library(cattonum): there is no package called 'cattonum'


### (1) 只使用高相關性特徵，模型表現反而不好。
### (2) 原始特徵多增加一組新的高相關性特徵之線性組合，模型表現沒有明顯變好。